In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## ChatOpenAI

OpenAI 사의 채팅 전용 Large Language Model(llm) 입니다.

객체를 생성할 때 다음을 옵션 값을 지정할 수 있습니다. 옵션에 대한 상세 설명은 다음과 같습니다.

> temperature

사용할 샘플링 온도는 0과 2 사이에서 선택합니다. 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.

> max_tokens

채팅 완성에서 생성할 토큰의 최대 개수입니다.

> model_name: 적용 가능한 모델 리스트

링크: https://platform.openai.com/docs/models

In [2]:
from langchain.chat_models import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 뭐야?"

# 질의
print(f"[답변]: {llm.predict(question)}")


c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[답변]: 대한민국의 수도는 서울이야.


### 프롬프트 템플릿의 활용

__PromptTemplate__

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다

[사용법]
- template: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 {}는 변수를 나타냅니다.
- input_variables: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

__input_variables__

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

[사용법] 
- 리스트 형식으로 변수 이름을 정의합니다.

In [3]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{country}의 수도는 뭐야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template=template)
prompt


PromptTemplate(input_variables=['country'], template='{country}의 수도는 뭐야?')

### LLMChain 객체

__LLMChain__

- LLMChain은 특정 PromptTemplate와 연결된 체인 객체를 생성합니다

[사용법]
- prompt: 앞서 정의한 PromptTemplate 객체를 사용합니다.
- llm: 언어 모델을 나타내며, 이 예시에서는 이미 어딘가에서 정의된 것으로 보입니다.

- run() 함수로 템플릿 프롬프트 실행
- predict() 함수로 run() 함수를 대체할 수 있습니다.
- apply() 함수로 여러개의 입력에 대한 처리를 한 번에 수행할 수 있습니다.
- generate() 는 문자열 대신에 LLMResult를 반환하는 점을 제외하고는 apply와 유사합니다.

In [4]:
from langchain.chains import LLMChain

# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 이전 CH02에서 chain 생성과 같음 
# chain = prompt | llm
# chain.invoke("일본").content

c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [5]:
# 체인 실행: run()
llm_chain.run(country="일본")

# 이전 CH02에서 chain.invoke 와 같음 
# chain = prompt | llm
# chain.invoke("일본").content

c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'일본의 수도는 도쿄입니다.'

In [8]:
llm_chain.predict(country="캐나다")

'캐나다의 수도는 오타와(Ottawa)입니다.'

In [9]:
# apply() 함수로 여러개의 입력에 대한 처리를 한 번에 수행할 수 있습니다.

input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

llm_chain.apply(input_list)

[{'text': '호주의 수도는 캔버라입니다.'},
 {'text': '중국의 수도는 베이징(北京)입니다.'},
 {'text': '네덜란드의 수도는 암스테르담입니다.'}]

In [10]:
# input_list 에 대한 결과 반환
result = llm_chain.apply(input_list)

# 반복문으로 결과 출력
for res in result:
    print(res["text"].strip())

호주의 수도는 캔버라입니다.
중국의 수도는 베이징(北京)입니다.
네덜란드의 수도는 암스테르담입니다.


In [11]:
# generate() 는 문자열 대신에 LLMResult를 반환하는 점을 제외하고는 apply와 유사합니다.
# LLMResult는 토큰 사용량과 종료 이유와 같은 유용한 생성 정보를 자주 포함하고 있습니다.

# 입력값
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

# input_list 에 대한 결과 반환
generated_result = llm_chain.generate(input_list)
print(generated_result)

generations=[[ChatGeneration(text='호주의 수도는 캔버라입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='호주의 수도는 캔버라입니다.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 17, 'total_tokens': 30}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cf620818-afe6-4564-a3bf-81c55bc8bf07-0'))], [ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 19, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a02344d1-eb69-4ac9-9178-4771bc079443-0'))], [ChatGeneration(text='네덜란드의 수도는 암스테르담입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='네덜란드의 수도는 암스테르담입니다.', response_metadata={

In [12]:
# 답변 출력
generated_result.generations

[[ChatGeneration(text='호주의 수도는 캔버라입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='호주의 수도는 캔버라입니다.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 17, 'total_tokens': 30}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cf620818-afe6-4564-a3bf-81c55bc8bf07-0'))],
 [ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 19, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a02344d1-eb69-4ac9-9178-4771bc079443-0'))],
 [ChatGeneration(text='네덜란드의 수도는 암스테르담입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='네덜란드의 수도는 암스테르담입니다.', response_metadata={'token_usa

In [19]:
# 답변 출력
for gen in generated_result.generations:
    print(gen[0].text.strip())

호주의 수도는 캔버라입니다.
중국의 수도는 베이징(北京)입니다.
네덜란드의 수도는 암스테르담입니다.


In [17]:
# 토큰 사용량 출력
generated_result.llm_output

{'token_usage': {'completion_tokens': 53,
  'prompt_tokens': 58,
  'total_tokens': 111},
 'model_name': 'gpt-3.5-turbo'}

In [18]:
# run ID 출력
generated_result.run

[RunInfo(run_id=UUID('cf620818-afe6-4564-a3bf-81c55bc8bf07')),
 RunInfo(run_id=UUID('a02344d1-eb69-4ac9-9178-4771bc079443')),
 RunInfo(run_id=UUID('7f1aa3f2-9560-4de5-ab53-1a29ad6347e6'))]

## 2개 이상의 변수를 템플릿 안에 정의

2개 이상의 변수를 적용하여 템플릿을 생성할 수 있습니다.

이번에는 2개 이상의 변수(input_variables) 를 활용하여 템플릿 구성을 해보겠습니다.

In [20]:
# 질문 템플릿 형식 정의
template = "{area1} 와 {area2} 의 시차는 몇시간이야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['area1', 'area2'], template='{area1} 와 {area2} 의 시차는 몇시간이야?')

In [21]:
# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [22]:
# 체인 실행: run()
print(llm_chain.run(area1="서울", area2="파리"))


서울과 파리의 시차는 8시간입니다. 서울은 GMT+9 시간대에 위치하고, 파리는 GMT+1 시간대에 위치하기 때문에 시차가 8시간입니다.


In [23]:
# 질문 템플릿 형식 정의
template = "{area1} 와 {area2} 의 시차는 몇시간이야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template)

# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 입력 리스트 
input_list = [
    {"area1": "파리", "area2": "뉴욕"},
    {"area1": "서울", "area2": "하와이"},
    {"area1": "켄버라", "area2": "베이징"},
]

# 반복문으로 결과 출력
result = llm_chain.apply(input_list)
for res in result:
    print(res["text"].strip())


파리와 뉴욕의 시차는 6시간입니다. 파리는 그리니치 표준시(GMT+1)를 따르고, 뉴욕은 동부 표준시(EST 또는 GMT-5)를 따르기 때문에 시차가 발생합니다.
서울과 하와이의 시차는 19시간입니다. 서울은 GMT+9 시간대에 속해 있고, 하와이는 GMT-10 시간대에 속해 있기 때문입니다.
켄버라와 베이징의 시차는 2시간입니다. 켄버라는 UTC+10 시간대에 속하고, 베이징은 UTC+8 시간대에 속합니다.


### 스트리밍(streaming)

스트리밍 옵션은 질의에 대한 답변을 실시간으로 받을 때 유용합니다.

다음과 같이 streaming=True 로 설정하고 스트리밍으로 답변을 받기 위한 StreamingStdOutCallbackHandler() 을 콜백으로 지정합니다.

In [24]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


In [25]:
# 질의내용
question = "대한민국에 대해서 300자 내외로 최대한 상세히 알려줘"

# 스트리밍으로 답변 출력
response = llm.predict(question)

대한민국은 동아시아에 위치한 고도 발전한 선진국가로, 수도는 서울이며 인구는 약 5천만 명이 넘습니다. 대한민국은 민주공화국으로 정치체제를 갖추고 있으며, 대통령이 국가의 수장이며 국회의원들이 국민을 대표하여 법을 제정하고 정부를 감독합니다.

대한민국은 세계에서 세 번째로 큰 경제 규모를 가지고 있으며, 선진국가로서 고도의 기술력과 경제력을 보유하고 있습니다. 주요 산업은 반도체, 자동차, 조선, 철강, IT 등이 있으며, 세계적으로 유명한 기업들이 많이 발달해 있습니다.

문화적으로는 한류가 세계적으로 유명하며, 한국의 전통문화인 한복, 한국음식, 불교, 기독교 등이 널리 알려져 있습니다. 또한 한국은 아름다운 자연환경과 문화유산을 보유하고 있어 많은 관광객들이 방문하는 인기 있는 여행지입니다.

한반도 북쪽에는 북한이 위치해 있으며, 남한과 북한은 아직까지 공식적으로 종전을 선언하지 않은 상태로 한반도의 평화를 위해 노력하고 있습니다. 대한민국은 미국, 일본, 중국 등과의 외교관계를 유지하며 국제사회에서 활발한 역할을 하고 있습니다.